---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [1]:
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
#import seaborn as sbn
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, roc_auc_score, auc
from sklearn.model_selection import train_test_split

def blight_model():

    data=pd.read_csv('train.csv', encoding='latin1', dtype={'grafitti_status':'str', 'non_us_str_code': 'str', 'zip_code': 'str'})
    t_data=pd.read_csv('test.csv')


    train_data=data.drop(['hearing_date','payment_amount', 'payment_date', 'payment_status', 'balance_due', 'collection_status',  
                      'disposition','admin_fee', 'state_fee', 'discount_amount', 'clean_up_cost', 'grafitti_status', 
                      'compliance_detail', 'violation_description', 'ticket_issued_date', 'non_us_str_code', 'country',
                      'violation_street_number','violation_street_name', 'violation_zip_code', 'mailing_address_str_number',
                      'mailing_address_str_name' ], axis=1)
#print(train_data)

    test_data=t_data.drop(['violation_street_number', 'violation_street_name','violation_zip_code', 'mailing_address_str_number',
                       'mailing_address_str_name', 'non_us_str_code', 'country', 'ticket_issued_date', 
                       'hearing_date', 'violation_code', 'violation_description', 'disposition',  'admin_fee', 'state_fee',
                       'clean_up_cost', 'grafitti_status'], axis=1)

#print(test_data)


    test_data.set_index("ticket_id", inplace=True)
    train_data.set_index('ticket_id', inplace=True)

    train_data.dropna(subset= ['compliance'], axis=0, inplace=True)
#test_data.dropna(subset= ['compliance'], axis=0, inplace=True)

    train_data['city']=train_data['city'].str.capitalize()
    test_data['city']=test_data['city'].str.capitalize()

    train_data["violator_name"]=train_data["violator_name"].fillna('not available')
    train_data['inspector_total_fines'] = train_data.groupby(by='inspector_name')['inspector_name'].transform('count')
    train_data['inspector_total_fines'].astype('int64')
    train_data['violator_total_fines'] = train_data.groupby(by='violator_name')['violator_name'].transform('count')
    train_data['violator_total_fines'].astype('int64')

    test_data["violator_name"]=test_data["violator_name"].fillna('not available')
    test_data['inspector_total_fines'] = test_data.groupby(by='inspector_name')['inspector_name'].transform('count')
    test_data['inspector_total_fines'].astype('int64')
    test_data['violator_total_fines'] = test_data.groupby(by='violator_name')['violator_name'].transform('count')
    test_data['violator_total_fines'].astype('int64')


    train_dummies = pd.get_dummies(train_data, columns=["agency_name"])
    train_dummies= train_dummies.rename(columns={'agency_name_Buildings, Safety Engineering & Env Department': 'Buildings, Safety Engineering & Env Department', 
                                      'agency_name_Department of Public Works': 'Department of Public Works', 'agency_name_Detroit Police Department': 'Detroit Police Department', 
                                      'agency_name_Health Department': 'Health Department', 'agency_name_Neighborhood City Halls':
                                        'Neighborhood City Halls'})
    train_dummies=train_dummies.drop(['Neighborhood City Halls'], axis=1)

    train_new=train_dummies[['fine_amount','inspector_total_fines', 'violator_total_fines',
        'Buildings, Safety Engineering & Env Department',
       'Department of Public Works', 'Detroit Police Department', 'late_fee', 'judgment_amount',
       'compliance']]

    test_dummies = pd.get_dummies(test_data, columns=["agency_name"])
    test_dummies= test_dummies.rename(columns={'agency_name_Buildings, Safety Engineering & Env Department': 'Buildings, Safety Engineering & Env Department', 
                                      'agency_name_Department of Public Works': 'Department of Public Works', 'agency_name_Detroit Police Department': 'Detroit Police Department', 
                                      'agency_name_Health Department': 'Health Department'})

    test_new=test_dummies[['fine_amount','inspector_total_fines', 'violator_total_fines',
        'Buildings, Safety Engineering & Env Department','Department of Public Works', 'Detroit Police Department', 'late_fee', 'judgment_amount']]


    X = train_new.iloc[:,:-1]
    y = train_new.iloc[:,-1]
    X_test=test_new

#print(X.shape)
#print(y)
#print(X_test.index)

#X_train, y_train = train_test_split(X, y, random_state = 0)
    
    gbdt = GradientBoostingClassifier(n_estimators= 100,learning_rate = 0.25, max_depth = 4, random_state = 0).fit(X,y)
    p=gbdt.predict(X_test)
#print(p)

    pred = gbdt.predict_proba(X_test)

#print(pred[:,1])
      
#df_series= pd.Series(pred[:,1])
#print(df_series)
    ds=pd.Series(pred[:,1], index=X_test.index)
    

# Gradient Boosting Clasifier (chosen after evaluations of different models, as best performing model)
# data series containing the ticket_id as index and the probability that each corresponding ticket from readonly/test.csv will be paid
    return ds

In [2]:
blight_model()

ticket_id
284932    0.074905
285362    0.032128
285361    0.071389
285338    0.114635
285346    0.080737
285345    0.070928
285347    0.077068
285342    0.576035
285530    0.031335
284989    0.053195
285344    0.077068
285343    0.050227
285340    0.026310
285341    0.025767
285349    0.094473
285348    0.083147
284991    0.036383
285532    0.039982
285406    0.048207
285001    0.021023
285006    0.019342
285405    0.007128
285337    0.032825
285496    0.044244
285497    0.040367
285378    0.029102
285589    0.054510
285585    0.090583
285501    0.046159
285581    0.029102
            ...   
376367    0.019164
376366    0.027892
376362    0.027892
376363    0.043871
376365    0.019164
376364    0.027892
376228    0.135355
376265    0.047780
376286    0.394076
376320    0.112120
376314    0.112914
376327    0.336685
376385    0.362649
376435    0.689454
376370    0.566165
376434    0.077697
376459    0.039839
376478    0.003590
376473    0.135355
376484    0.040559
376482    0.018352
37